In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
from imblearn.datasets import fetch_datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.io import arff
import warnings
warnings.filterwarnings("ignore")

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
%matplotlib inline

In [3]:
data = arff.loadarff('/Users/Bjoern/Desktop/Summer/Work on Datasets/Training Dataset.arff')
df = pd.DataFrame(data[0])
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Bjoern/Desktop/Summer/Work on Datasets/Training Dataset.arff'

In [ ]:
# inting dataset
for i in df:
    df.loc[:,i] = df.loc[:,i].astype(int)
df.head()

In [ ]:
# mean, sdv, etc.
df.describe()

In [ ]:
# check for null vals
df.isnull().sum().max()

In [ ]:
# list features
df.columns 

In [ ]:
# distribution of Phising:Not Phishing
print("Phishing: " + str(round(len(df[df['Result'] == 1])/len(df) * 100,2)) + "%")
print("Not Phishing: " + str(round(len(df[df['Result'] == -1])/len(df) * 100,2)) + "%")


In [ ]:
# graphed version of distribution
colors = ["#0101DF", "#DF0101"]

sns.countplot('Result', data=df, palette=colors)
plt.title('Result Distributions \n (-1: No Phising || 1: Phising)', fontsize=14)

In [ ]:
X = df.drop('Result',axis = 1)
y = df['Result']

In [ ]:
# Produce a histgram of the data vs how many there are

# There probably is a better way to do this but I haven't figured it out

fig, ax = plt.subplots(6,5,figsize = (30,30))
ax = ax.flatten()

having_IP_Address = df['having_IP_Address'].values
URL_Length = df['URL_Length'].values
Shortining_Service = df['Shortining_Service'].values
having_At_Symbol = df['having_At_Symbol'].values
double_slash_redirecting = df['double_slash_redirecting'].values
Prefix_Suffix = df['Prefix_Suffix'].values
having_Sub_Domain = df['having_Sub_Domain'].values
SSLfinal_State = df['SSLfinal_State'].values
Domain_registeration_length = df['Domain_registeration_length'].values
Favicon = df['Favicon'].values
port = df['port'].values
HTTPS_token = df['HTTPS_token'].values
Request_URL = df['Request_URL'].values
URL_of_Anchor = df['URL_of_Anchor'].values
Links_in_tags = df['Links_in_tags'].values
SFH = df['SFH'].values
Submitting_to_email = df['Submitting_to_email'].values
Abnormal_URL = df['Abnormal_URL'].values
Redirect = df['Redirect'].values
on_mouseover = df['on_mouseover'].values
RightClick = df['RightClick'].values
popUpWidnow = df['popUpWidnow'].values
Iframe = df['Iframe'].values
age_of_domain = df['age_of_domain'].values
DNSRecord = df['DNSRecord'].values
web_traffic = df['web_traffic'].values
Page_Rank = df['Page_Rank'].values
Google_Index = df['Google_Index'].values
Links_pointing_to_page = df['Links_pointing_to_page'].values
Statistical_report = df['Statistical_report'].values

array_names = ['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL',
       'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe',
       'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank',
       'Google_Index', 'Links_pointing_to_page', 'Statistical_report']


array_vars = [having_IP_Address, URL_Length, Shortining_Service,
       having_At_Symbol, double_slash_redirecting, Prefix_Suffix,
       having_Sub_Domain, SSLfinal_State, Domain_registeration_length,
       Favicon, port, HTTPS_token, Request_URL, URL_of_Anchor,
       Links_in_tags, SFH, Submitting_to_email, Abnormal_URL,
       Redirect, on_mouseover, RightClick, popUpWidnow, Iframe,
       age_of_domain, DNSRecord, web_traffic, Page_Rank,
    Google_Index, Links_pointing_to_page, Statistical_report]

for i in range(len(array_names)):
    sns.distplot(array_vars[i], ax = ax[i])
    ax[i].set_title(array_names[i], fontsize=14)
    ax[i].set_xlim([min(array_vars[i]), max(array_vars[i])])

plt.show()

In [ ]:
# Plot the correlation matrix between each feature
fig, ax = plt.subplots(figsize=(14,8))

corr = df.corr()
sns.heatmap(corr, cmap='coolwarm_r',annot_kws={'size':20})
ax.set_title('Correlation Matrix', fontsize =14)

In [ ]:
# PCA/TSNE, 2D visualisation

# TSNE (also takes long time > 5 mins), maybe try MultiTSNE but need to install a bunch of different packages
X_reduced_tsne = TSNE(n_components=2, random_state=42).fit_transform(X.values)

# PCA Implementation
X_reduced_pca = PCA(n_components=2, random_state=42).fit_transform(X.values)

# TruncatedSVD
X_reduced_svd = TruncatedSVD(n_components=2, algorithm='randomized', random_state=42).fit_transform(X.values)

In [ ]:
# plot reduced data

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24,6))
# labels = ['No Fraud', 'Fraud']
f.suptitle('Clusters using Dimensionality Reduction', fontsize=14)


blue_patch = mpatches.Patch(color='#0A0AFF', label='Not Phishing')
red_patch = mpatches.Patch(color='#AF0000', label='Phishing')


# t-SNE scatter plot
ax1.scatter(X_reduced_tsne[:,0], X_reduced_tsne[:,1], c=(y == -1), cmap='coolwarm', label='No Phishing', linewidths=2)
ax1.scatter(X_reduced_tsne[:,0], X_reduced_tsne[:,1], c=(y == 1), cmap='coolwarm', label='Phishing', linewidths=2)
ax1.set_title('t-SNE', fontsize=14)

ax1.grid(True)

ax1.legend(handles=[blue_patch, red_patch])

# PCA scatter plot
ax2.scatter(X_reduced_pca[:,0], X_reduced_pca[:,1], c=(y == -1), cmap='coolwarm', label='No Phishing', linewidths=2)
ax2.scatter(X_reduced_pca[:,0], X_reduced_pca[:,1], c=(y == 1), cmap='coolwarm', label='Phishing', linewidths=2)
ax2.set_title('PCA', fontsize=14)

ax2.grid(True)

ax2.legend(handles=[blue_patch, red_patch])

# TruncatedSVD scatter plot
ax3.scatter(X_reduced_svd[:,0], X_reduced_svd[:,1], c=(y == -1), cmap='coolwarm', label='No Phishing', linewidths=2)
ax3.scatter(X_reduced_svd[:,0], X_reduced_svd[:,1], c=(y == 1), cmap='coolwarm', label='Phishing', linewidths=2)
ax3.set_title('Truncated SVD', fontsize=14)

ax3.grid(True)

ax3.legend(handles=[blue_patch, red_patch])

plt.show()

In [ ]:
# Split test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# PCA, not recommended but this is just to see how dimensionality reduction would work
components = 29
while True: # make sure it runs at least once
    X_pca = PCA(n_components = components).fit(X.values) 
    if (X_pca.noise_variance_ > 0.1):
        break
    components -= 1
components += 1  # we end with variance less than 90%, so add 1 to n_components to get > 90%
X_pca = PCA(n_components = components).fit(X.values)  # X_pca is the reduced feature 
print(X_pca.noise_variance_, components)

In [ ]:
# Recursive feature elimination

from sklearn.feature_selection import RFE

model = LogisticRegression()

rfe = RFE(model, n_features_to_select = 16)
rfe = rfe.fit(X_train,y_train)

print(rfe.support_)
print(rfe.ranking_)
print(rfe.score(X_train,y_train))

X_reduced = rfe.transform(X_train)
y_pred = rfe.predict(X_train)
print(f1_score(y_train,y_pred))

In [ ]:
# plot histogram of importance

plt.figure(figsize=(10,8),dpi = 80)
plt.bar(range(1,31),rfe.ranking_)
plt.show()

In [ ]:
# Whats the best
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score


training_score = cross_val_score(model, X_train, y_train, cv=5)
print(round(training_score.mean(),5))

print(round(cross_val_score(model, X_reduced, y_train, cv=5).mean(),5))